# 1.4: Generate WEPP Target Variable via WEPPcloud API

**Objective:** To programmatically execute a WEPP simulation using the WEPPcloud web service, monitor its progress, and download the final raster of average annual sediment yield. This notebook completely replaces the manual workflow of downloading climate and soil data.

### 1. Setup and Configuration

* **Imports:** Import necessary libraries (`wepppy`, `pathlib`, `yaml`, etc.).
* **Load Config:** Load the `weppcloud` section from the main `config.yml` file.
* **Load AOI:** Read the study area shapefile specified in the configuration.

In [1]:
from __future__ import annotations

import logging
import math
import re
import sys
from pathlib import Path
from shapely.geometry import box
import yaml
import geopandas as gpd
import pandas as pd
import psutil
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib.lines import Line2D
import contextily as ctx

# --- Project-Specific Modules ---
# Add project's src directory to path to allow imports
def find_project_root(marker='config.yml'):
    path = Path.cwd().resolve()
    while path.parent != path:
        if (path / marker).exists():
            return path
        path = path.parent
    raise FileNotFoundError(f"Project root with marker '{marker}' not found.")

PROJECT_ROOT = find_project_root()
if str(PROJECT_ROOT / 'src') not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT / 'src'))

from geo_tda.utils import setup_colored_logging
from geo_tda.data_acquisition.weppcloud_client import (
    submit_weppcloud_job,
    monitor_job_status,
    download_weppcloud_result
)
# from src.plotting_utils import plot_raster_with_boundary

setup_colored_logging()
log = logging.getLogger("1.4_generate_target_variable")

# Load configuration
CONFIG_PATH = PROJECT_ROOT / "config.yml"
with open(CONFIG_PATH) as f:
    config = yaml.safe_load(f)

W_CONFIG = config['weppcloud']
log.info("✅ Configuration loaded successfully.")

# Resolve paths from config
AOI_PATH = PROJECT_ROOT / W_CONFIG['aoi_path']
OUTPUT_DIR = PROJECT_ROOT / W_CONFIG['output_directory']

log.info(f"Study Area Shapefile: {AOI_PATH}")
log.info(f"Output Directory: {OUTPUT_DIR}")

2025-10-19 22:18:08 - 1.4_generate_target_variable - INFO - ✅ Configuration loaded successfully.
2025-10-19 22:18:08 - 1.4_generate_target_variable - INFO - Study Area Shapefile: D:\OneDrive - University of Georgia\multiscale_tda_geomorphology\data\raw\boundaries\study_provinces.shp
2025-10-19 22:18:08 - 1.4_generate_target_variable - INFO - Output Directory: D:\OneDrive - University of Georgia\multiscale_tda_geomorphology\data\processed\target_variable


### 2. Call `weppcloud_client.py` Functions


In [2]:
job_id = None
job_successful = False
final_raster_path = None

try:
    # --- Step 1: Submit Job ---
    job_id = submit_weppcloud_job(
        aoi_path=AOI_PATH,
        project_name=W_CONFIG['project_name'],
        simulation_years=W_CONFIG['simulation_years']
    )

    # --- Step 2: Monitor Status ---
    if job_id:
        # Using a shorter poll interval for this demonstration
        job_successful = monitor_job_status(job_id, poll_interval_sec=30)
    else:
        raise RuntimeError("Failed to submit job; could not retrieve a valid Job ID.")

    # --- Step 3: Download Result ---
    if job_successful:
        final_raster_path = download_weppcloud_result(
            job_id=job_id,
            output_dir=OUTPUT_DIR
        )
    else:
        log.error("WEPPcloud job did not complete successfully. Skipping result download.")

except Exception as e:
    log.critical(f"An unexpected error occurred during the workflow: {e}", exc_info=True)

2025-10-19 22:18:08 - geo_tda.data_acquisition.weppcloud_client - INFO - 🚀 Submitting new WEPPcloud job: 'Multiscale_TDA_Geomorphology_Fall2025'
2025-10-19 22:18:08 - geo_tda.data_acquisition.weppcloud_client - INFO -    - AOI Shapefile: D:\OneDrive - University of Georgia\multiscale_tda_geomorphology\data\raw\boundaries\study_provinces.shp
2025-10-19 22:18:08 - geo_tda.data_acquisition.weppcloud_client - INFO -    - Simulation Length: 100 years
2025-10-19 22:18:08 - geo_tda.data_acquisition.weppcloud_client - INFO - ✅ Job successfully submitted. Job ID: job_a132b5b48f7c
2025-10-19 22:18:08 - geo_tda.data_acquisition.weppcloud_client - INFO - ⏳ Monitoring status for Job ID: job_a132b5b48f7c (checking every 30s)
2025-10-19 22:18:08 - geo_tda.data_acquisition.weppcloud_client - INFO -    - Current status: PENDING (Elapsed: 0s)
2025-10-19 22:18:38 - geo_tda.data_acquisition.weppcloud_client - INFO -    - Current status: RUNNING (Elapsed: 30s)
2025-10-19 22:19:08 - geo_tda.data_acquisition

WEPPcloud Result:   0%|          | 0/1 [00:00<?, ?file/s]

2025-10-19 22:20:45 - geo_tda.data_acquisition.download_core - ERROR - Download failed for sediment_yield.tif: RetryError[<Future at 0x1e7f3059bd0 state=finished raised HTTPError>]
2025-10-19 22:20:45 - geo_tda.data_acquisition.download_core - INFO - ======================================================================
2025-10-19 22:20:45 - geo_tda.data_acquisition.download_core - INFO - Download Summary for WEPPcloud Result:
2025-10-19 22:20:45 - geo_tda.data_acquisition.download_core - INFO -   Downloaded: 0
2025-10-19 22:20:45 - geo_tda.data_acquisition.download_core - INFO -   Skipped:    0 (valid files already exist)
2025-10-19 22:20:45 - geo_tda.data_acquisition.download_core - INFO -   Failed:     1 (includes validation failures)
2025-10-19 22:20:45 - geo_tda.data_acquisition.download_core - INFO - ======================================================================
2025-10-19 22:20:45 - geo_tda.data_acquisition.download_core - ERROR - --- Download Failures ---
2025-10-19 22:

### 3. Post-Flight Validation

* **Verify File:** Check that the downloaded raster exists and is not empty.
* **Check CRS and Extent:** Use `rasterio` to confirm that the output raster's projection and grid are consistent with the project's source DEM.
* **Visualize:** Display the final raster on a map to confirm it covers the correct geographic area.

In [3]:
log.info("--- Workflow Complete: Final Status ---")

if final_raster_path and final_raster_path.exists():
    log.info(f"✅ SUCCESS: The target variable raster has been generated and saved to:")
    log.info(f"   -> {final_raster_path}")

    # --- 1. Verify Raster Metadata ---
    log.info("--- Verifying Raster Properties ---")
    try:
        with rasterio.open(final_raster_path) as src:
            log.info(f"   - Raster CRS: {src.crs}")
            log.info(f"   - Raster Dimensions: {src.width}x{src.height}")
            log.info(f"   - Raster NoData Value: {src.nodata}")
            log.info("✅ Raster metadata is readable and valid.")
    except Exception as e:
        log.error(f"❌ Failed to read raster metadata: {e}")

    # --- 2. Visualize the Final Output ---
    log.info("--- Generating Final Visualization ---")

    # Load the study area boundary for the plot
    aoi_gdf = gpd.read_file(AOI_PATH)

    fig, ax = plt.subplots(figsize=(12, 12), dpi=150)

    plot_raster_with_boundary(
        raster_path=final_raster_path,
        boundary_gdf=aoi_gdf,
        ax=ax,
        cmap='viridis',
        title="WEPPcloud Average Annual Sediment Yield"
    )

    # Save the figure to the reports directory
    REPORTS_DIR = PROJECT_ROOT / config['project_paths']['reports']
    figure_path = REPORTS_DIR / "weppcloud_sediment_yield_map.png"
    fig.savefig(figure_path, dpi=300, bbox_inches='tight', pad_inches=0.1)
    log.info(f"✅ Final map saved to: {figure_path}")

    plt.show()

else:
    log.error("❌ FAILURE: The workflow did not produce the final target raster.")
    if not job_successful:
        log.error("\t- Reason: The WEPPcloud simulation failed or was cancelled.")
    else:
        log.error("\t- Reason: The result download failed after a successful simulation.")

2025-10-19 22:20:45 - 1.4_generate_target_variable - INFO - --- Workflow Complete: Final Status ---
2025-10-19 22:20:45 - 1.4_generate_target_variable - ERROR - ❌ FAILURE: The workflow did not produce the final target raster.
2025-10-19 22:20:45 - 1.4_generate_target_variable - ERROR - 	- Reason: The result download failed after a successful simulation.
